In [ ]:
from MT5Connector import MT5Connector
from CandlestickPatterns import CandlestickPatterns
import MetaTrader5 as mt5
import pandas as pd
import time
from RiskManager import RiskManager


capital = MT5Connector.fetch_account_balance()


def has_open_trade(symbol):
    open_positions = mt5.positions_get(symbol=symbol)
    if open_positions is None:
        print("Failed to get open positions.")
        return False
    return len(open_positions) > 0



def place_order(symbol, order_type, stop_loss_distance, take_profit_distance):
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"{symbol} not found, cannot place order.")
        return
    
    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Failed to select {symbol}.")
            return
    
    entry_price = mt5.symbol_info_tick(symbol).ask if order_type == "BUY" else mt5.symbol_info_tick(symbol).bid
    point = symbol_info.point
    
    if order_type == "BUY":
        sl_price = entry_price - stop_loss_distance * point
        tp_price = entry_price + take_profit_distance * point
    else:  # For SELL orders
        sl_price = entry_price + stop_loss_distance * point
        tp_price = entry_price - take_profit_distance * point

    risk_manager = RiskManager(capital, entry_price, tp_price, sl_price, 5)
    lot_size = risk_manager.calculate_lot_size(symbol)
    print (lot_size)
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot_size,
        "type": mt5.ORDER_TYPE_BUY if order_type == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "sl": sl_price,
        "tp": tp_price,
        "magic": 1000,
        "comment": "Automated script trade",
        "type_time": mt5.ORDER_TIME_GTC,  # Good till cancel
        "type_filling": mt5.ORDER_FILLING_IOC,  # Immediate or cancel
    }
    
    result = mt5.order_send(request)
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Order failed, retcode={result.retcode}")
    else:
        print(f"Order placed successfully, {order_type}, {lot_size} lots of {symbol}")


        

def analyze_timeframe(df, timeframe_name):
    results = {
        "bullish": [],
        "bearish": [],
        "neutral": []
    }

    # Single candle patterns
    if CandlestickPatterns.is_doji(df.iloc[-1]):
        results["neutral"].append("Doji")
    if CandlestickPatterns.is_hammer(df.iloc[-1]):
        results["bullish"].append("Hammer")
    if CandlestickPatterns.is_inverted_hammer(df.iloc[-1]):
        results["bullish"].append("Inverted Hammer")
    if CandlestickPatterns.is_hanging_man(df.iloc[-1]):
        results["bearish"].append("Hanging Man")
    if CandlestickPatterns.is_shooting_star(df.iloc[-1]):
        results["bearish"].append("Shooting Star")

    # Patterns requiring the last 2 candles
    if len(df) >= 2:
        if CandlestickPatterns.is_bullish_engulfing(df.iloc[-1], df.iloc[-2]):
            results["bullish"].append("Bullish Engulfing")
        if CandlestickPatterns.is_bearish_engulfing(df.iloc[-1], df.iloc[-2]):
            results["bearish"].append("Bearish Engulfing")
        if CandlestickPatterns.is_bullish_harami(df.iloc[-1], df.iloc[-2]):
            results["bullish"].append("Bullish Harami")
        if CandlestickPatterns.is_bearish_harami(df.iloc[-1], df.iloc[-2]):
            results["bearish"].append("Bearish Harami")


        for i in range(2, len(df)): 
            chunk = df.iloc[i-2:i+1]
            local_index = 2  
    
            if CandlestickPatterns.is_morning_star(chunk, local_index):
                results["bullish"].append("Morning Star")
            if CandlestickPatterns.is_evening_star(chunk, local_index):
                results["bearish"].append("Evening Star")
            if CandlestickPatterns.is_three_white_soldiers(chunk, local_index):
                results["bullish"].append("Three White Soldiers")
            if CandlestickPatterns.is_three_black_crows(chunk, local_index):
                results["bearish"].append("Three Black Crows")
    return results

def analyze_and_trade(symbol, analysis_results):
    bullish_patterns = {}
    bearish_patterns = {}

   
    for timeframe, patterns in analysis_results.items():
        bullish_patterns[timeframe] = patterns["bullish"]
        bearish_patterns[timeframe] = patterns["bearish"]
    

    bullish_count = sum(len(bullish_patterns[tf]) > 0 for tf in bullish_patterns)
    bearish_count = sum(len(bearish_patterns[tf]) > 0 for tf in bearish_patterns)
    

    if has_open_trade(symbol):
        print(f"Trade skipped: There is already an existing trade for {symbol}.")
        return

    if bullish_count >= 2:
        print(f"Placing BUY order for {symbol} based on majority bullish sentiment in {bullish_count} timeframes.")
        price = mt5.symbol_info_tick(symbol).ask  
        sl = 50  # Example SL in points
        tp = 100  # Example TP in points
        place_order(symbol, "BUY", sl, tp)
    elif bearish_count >= 2:
        print(f"Placing SELL order for {symbol} based on majority bearish sentiment in {bearish_count} timeframes.")
    
        price = mt5.symbol_info_tick(symbol).bid 
        sl = 50 
        tp = 100  
        place_order(symbol, "SELL", sl, tp)
    else:
        print("No decisive market sentiment across timeframes. No action taken.")



 

def fetch_and_analyze(symbol, timeframe):
    """Fetch the latest candle data for the specified timeframe and symbol and analyze it."""
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 20)  # Fetch more candles to accommodate patterns requiring more data
    # Check if rates is not None and has elements
    if rates is not None and len(rates) > 0:
        df = pd.DataFrame(rates)
        # Convert 'time' to a readable format
        df['time'] = pd.to_datetime(df['time'], unit='s')
        return analyze_timeframe(df, timeframe)  # Adjusted to directly analyze the dataframe
    else:
        print(f"Failed to fetch rates for {symbol} in {timeframe} timeframe.")
        return {
            "bullish": [],
            "bearish": [],
            "neutral": []
        }


        
def analyze_all_timeframes(connector, symbol, timeframes):
    results = {}
    for timeframe_name, timeframe in timeframes.items():
        print(f"Analyzing {timeframe_name} timeframe for {symbol}...")
        patterns = fetch_and_analyze(symbol, timeframe)  # Pass symbol here
        results[timeframe_name] = patterns
    return results


if __name__ == "__main__":
    account = 69620949
    password = "123@Trader"
    server = "XMGlobal-MT5 2"
    symbol = "EURUSD"
    
    connector = MT5Connector(account, password, server)
    connector.initialize()

    timeframes = {
        "15M": mt5.TIMEFRAME_M15,
        "30M": mt5.TIMEFRAME_M30,
        "1H": mt5.TIMEFRAME_H1,
        "4H": mt5.TIMEFRAME_H4,
        "D1": mt5.TIMEFRAME_D1
    }


    
    while True:
        analysis_results = analyze_all_timeframes(connector, symbol, timeframes)
        for timeframe, result in analysis_results.items():
            bullish_patterns = ', '.join(set(result["bullish"]))  # Convert list to set to remove duplicates
            bearish_patterns = ', '.join(set(result["bearish"]))  # Convert list to set to remove duplicates
            if bullish_patterns or bearish_patterns:
                sentiment = "bullish" if len(result["bullish"]) > len(result["bearish"]) else "bearish"
                print(f"In the {timeframe} timeframe, sentiment is {sentiment} due to bullish patterns: {bullish_patterns} and bearish patterns: {bearish_patterns}")
            else:
                print(f"In the {timeframe} timeframe, no significant patterns found.")

        analyze_and_trade(symbol, analysis_results)
        print("Sleeping for next analysis cycle...")
        time.sleep(60)